# CoVid-19 Tracing

In [ ]:
#! mkdir data

In [ ]:
#%cd ./data
%cd ./data/COVID-19

In [ ]:
#!git clone https://github.com/CSSEGISandData/COVID-19.git
!git pull

In [ ]:
%cd ../..

##### We are going to use the data from CSSEGISandData 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from datetime import date, datetime, timedelta
import bokeh
import pandas_bokeh
from pandas_profiling import ProfileReport

### 1. Data acquisition

We must get the files from the source:

In [2]:
files=[]
# The path where the files are is:#
path = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'
#path = './data/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'
# We must get the name of each file. All the are dates, so:

start = start_date=date(2020, 1, 22)
end = date.today()
date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days)]

for dates in date_generated:
    dates=dates.strftime("%m-%d-%Y")+'.csv'
    files.append(dates)
    
def dataframe(files):
    try:
        datos=pd.DataFrame()
        for file in files:
            filepath=path+file
            df_file=pd.read_csv(filepath)
            datos=pd.concat([datos,df_file],axis=0)
        return datos
    except TimeoutError:
        return dataframe(files)
datos=dataframe(files)

https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-22-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-23-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-24-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-25-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-26-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-27-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-28-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports

In [4]:
df=datos.copy()

### Cleaning data

Data must be cleaned to get a better result. It should be checked periodically.

In [5]:
df.rename(columns={'Country/Region':'Country'},inplace=True)
df.rename(columns={'Province/State':'Province'},inplace=True)
# Some names must be cleaned manually
df.loc[df['Province']=='Fench Guiana','Province']='French Guiana'
df.loc[df['Province']=='Hong Kong','Country'] = 'Hong Kong'
df.loc[df['Province']=='Macau','Country'] = 'Macao'
df.loc[df['Province']=='Taiwan','Country'] = 'Taiwan'
df.loc[df['Province']=='UK','Province'] = np.NaN
df.loc[df['Country'].str.startswith('Tai'),'Country'] = 'Taiwan'
#Some of them are twice but with different names
list_unique_s=list(df['Province'].unique())
list_unique_s.remove(np.NaN)
for state in list_unique_s:
    filtro=df['Province'].str.contains(state).fillna(False)
    unicos=df.loc[filtro,'Province'].unique()
    if len(unicos)>1:
        df.loc[filtro,'Province']=unicos[0]

list_unique_c=list(df['Country'].unique())
for country in list_unique_c:
    filtro=df['Country'].str.contains(country).fillna(False)
    unicos=df.loc[filtro,'Country'].unique()
    if len(unicos)>1:
        df.loc[filtro,'Country']=unicos[0]
        
# In some rows the Country is placed in the 'Province/State' column
for i in range(df.shape[0]):
    if df.iloc[i,0]==df.iloc[i,1]:
        df.iloc[i,0]=np.NaN
    else:
        continue

C:\Users\joeld\Anaconda3\lib\site-packages\pandas\core\strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


#### Data Localization

In [6]:
df_aux=df.copy()
df_aux=df_aux.reset_index().drop('index',axis=1)

We may get the Address based on the coordimates:

In [11]:
df_aux['Address']=np.NaN

def locate(row):
    geolocator = Nominatim(user_agent="CoVid-19",timeout=10)
    if np.isnan(row['Latitude']):
        row['Address']=np.NaN
    else:
        lat=row['Latitude']
        lon=row['Longitude']
        row['Address']=geolocator.reverse(str(lat)+', '+str(lon))

df_aux.apply(lambda row: locate(row),axis=1)

GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

```python
df_aux['Address']=np.NaN

def locate(row):
    geolocator = Nominatim(user_agent="CoVid-19",timeout=3)
    try:
        if np.isnan(row['Latitude']):
            row['Address']=np.NaN
        else:
            lat=row['Latitude']
            lon=row['Longitude']
            row['Address']=geolocator.reverse(str(lat)+', '+str(lon))
    except GeocoderTimedOut:
            return GeocoderTimedOut

df_aux.apply(lambda row: locate(row),axis=1)
```